<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Jupyter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Dynamic systems: 8. Reactions
## Two reactants
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

----
In this notebook, we solve the continuity equation for **two reactants** with terms for:
- advection
- diffusion
- reaction
- source-sink

----
## 2D advection-diffusion-reaction equation

Check the interesting pages from [Karl Sims](http://www.karlsims.com/rd.html)
and [Robert Munafo](http://mrob.com/pub/comp/xmorphia).

We consider the transport equations for species $A$ and $B$:
$$
\begin{array}{rcl}
\frac{\partial A}{\partial t}
+ \vec{u} \cdot \nabla A
- \nabla \cdot (D \nabla A) 
&=& f_A - k_1 A B - \lambda_A A \\
\frac{\partial B}{\partial t}
+ \vec{u} \cdot \nabla B
- \nabla \cdot (D \nabla B)
&=& f_B + k_1 A B - \lambda_B B 
\end{array}
$$

We rewrite the transport equation into a finite difference scheme with explicit time
integration, using $\frac{\partial A}{\partial t}=\frac{A(t_{i+1})-A(t_i)}{\Delta t}$:
$$
\begin{array}{rcl}
A(t_{i+1}) = A(t_i) +
\left[
- \vec{u} \cdot \nabla A
+ \nabla \cdot (D \nabla A)
+ f_A - k_1 A B
- \lambda_A A
\right] \Delta t \\
B(t_{i+1}) = B(t_i) +
\left[
- \vec{u} \cdot \nabla B
+ \nabla \cdot (D \nabla B)
+ f_B + k_1 A B
- \lambda_B B
\right] \Delta t
\end{array}
$$
In the square brackets, we have the **advection term**, the **diffusion term**,
a **source-sink term**, and a **reaction term**.

### Reaction

We use for the two reactant exercise a special reaction, known as the **Gray-Scott model**.

One part of $A$ reacts with two parts of $B$ to form three parts of $B$:
$$
A B B \stackrel{k_1}{<->} B B B
$$
Reformulated as differential equation:
$$
\frac{dB}{dt} = k_1 A B B
$$
We set $k_1=1$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Set parameter values

## Create mesh

In [ ]:
def create_mesh(xmin,xmax,ymin,ymax,nx=11,ny=11):
    x = np.linspace(xmin,xmax,nx)
    y = np.linspace(ymin,ymax,ny)
    dx = x[1]-x[0]
    dy = y[1]-y[0]
    X,Y = np.meshgrid(x,y)
    return X,Y,dx,dy

## Initialize field values

In [ ]:
def init_fields(X,Y,velx,vely,nx,ny,shake=0.):
    # concentration A
    A = np.ones(nx*ny).reshape(ny,nx)
    B = np.zeros(nx*ny).reshape(ny,nx)
    A = (1-shake)*A + shake * np.random.random((ny,nx))
    B = (1-shake)*B + shake * np.random.random((ny,nx))
    xc = (X.min()+X.max()) / 2
    yc = (Y.min()+Y.max()) / 2
    rc = 10.
    print(xc,yc,rc)
    for i in range(ny):
        for j in range(nx):
            dist = np.sqrt((X[i,j]-xc)**2 + (Y[i,j]-yc)**2)
            if (dist <= rc):
                A[i,j] = 0.50
                B[i,j] = 0.25
    # velocity
    Vx = np.ones(nx*ny).reshape(ny,nx)
    Vy = np.ones(nx*ny).reshape(ny,nx)
    Vx = velx*Vx
    Vy = vely*Vy
    Vabs = np.sqrt(Vx**2 + Vy**2)
    return A,B,Vx,Vy,Vabs

In [ ]:
def plot_fields(X,Y,A,B,Vx,Vy,Vabs,filename):
    fig, axs = plt.subplots(2,1,figsize=(8,10))
    
    v = np.linspace(0.0, 1.0, 11, endpoint=True)
    #plt.contour(xi, yi, zi, v, linewidths=0.5, colors='k')
    #plt.contourf(xi, yi, zi, v, cmap=plt.cm.jet)
    #x = plt.colorbar(ticks=v)

    axs[0].set_xlim([xmin,xmax])
    axs[0].set_ylim([ymin,ymax])
    CS0=axs[0].contourf(X,Y,A,v, cmap=plt.cm.jet_r)
    axs[0].quiver(X,Y,Vx/Vabs,Vy/Vabs,Vabs,alpha=0.2,width=0.002,scale=150,pivot="middle")
    axs[0].quiver(X,Y,Vx/Vabs,Vy/Vabs, alpha=0.2,edgecolor='k', facecolor='None', 
              linewidth=.5,width=0.002,scale=150,pivot="middle")
    cbar = fig.colorbar(CS0,ax=axs[0],orientation="vertical",shrink=0.9)
    cbar.ax.set_ylabel('A [mol/m$^3$]')
    
    CS1=axs[1].contourf(X,Y,B,v, cmap=plt.cm.jet_r)
    cbar = fig.colorbar(CS1,ax=axs[1],orientation="vertical",shrink=0.9)
    cbar.ax.set_ylabel('B [mol/m$^3$]')
    
    axs[0].set_title('A')
    axs[1].set_title('B')
    #axs[0].axis('off')
    #axs[1].axis('off')
    plt.savefig('lab08/'+filename,dpi=300)
    plt.close()

## Define differential operators as finite-difference operators

In [ ]:
def diffusion_old(A,dx,dy):
    L = 0.*A
    for i in range(1,A.shape[0]-1):
        for j in range(1,A.shape[1]-1):
            L[i,j] = (A[i+1,j]+A[i-1,j]+A[i,j+1]+A[i,j-1]-4*A[i,j]) / (dx*dy)
    return L

def advection_old(A,Vx,Vy,dx,dy):
    L = 0.*A
    for i in range(1,A.shape[0]-1):
        for j in range(1,A.shape[1]-1):
            grady = (A[i+1,j]-A[i-1,j]) / (2*dy)
            gradx = (A[i,j+1]-A[i,j-1]) / (2*dx)
            L[i,j] = Vx[i,j]*gradx + Vy[i,j]*grady
    return L

In [ ]:
def diffusion(M,dx,dy):
    """L = nabla^2 M"""
    L = -4*M
    L += np.roll(M, (0,-1), (0,1)) # right neighbor
    L += np.roll(M, (0,+1), (0,1)) # left neighbor
    L += np.roll(M, (-1,0), (0,1)) # top neighbor
    L += np.roll(M, (+1,0), (0,1)) # bottom neighbor
    L = L / (dx*dy)
    return L

def advection(M,Vx,Vy,dx,dy):
    """L = V*nabla M"""
    gradx  = np.roll(M, (0,-1), (0,1)) # right neighbor
    gradx -= np.roll(M, (0,+1), (0,1)) # left neighbor
    gradx  = gradx / (2*dx)
    grady  = np.roll(M, (-1,0), (0,1)) # top neighbor
    grady -= np.roll(M, (+1,0), (0,1)) # bottom neighbor
    grady  = grady / (2*dy)
    L = Vx*gradx + Vy*grady
    return L

In [ ]:
def update_fields(A,B,Vx,Vy,DA,DB,lambA,lambB,f,k,dt,dx,dy):
    nx = A.shape[0]
    ny = A.shape[1]
    # diffusion
    laplacian = diffusion(A,dx,dy)
    gradient  = advection(A,Vx,Vy,dx,dy)
    # Now apply the update formula
    diff_A = (-gradient + DA*laplacian - lambA*A - A*B**2 + f*(1-A)) * dt
    # diffusion
    laplacian = diffusion(B,dx,dy)
    gradient  = advection(B,Vx,Vy,dx,dy)
    # Now apply the update formula
    diff_B = (-gradient + DB*laplacian - lambB*B + A*B**2 - (k+f)*B) * dt

    A += diff_A
    B += diff_B
    #print(A.min(),A.max())

    return A,B

---- 
## Examples

### 1

In [ ]:
xmin = 0. 
xmax = 200.
ymin = 0. 
ymax = 200.
nx   = 200
ny   = 200
DA   = 0.16
DB   = 0.08
lambA = 0.0
lambB = 0.0
f = 0.060
k = 0.062
velx = 0.0
vely = 0.
tmin = 0.
tmax = 10000.
dt   = 1.0
name = 'TWO-1-'

X,Y,dx,dy = create_mesh(xmin,xmax,ymin,ymax,nx,ny)
A,B,Vx,Vy,Vabs = init_fields(X,Y,velx,vely,nx,ny,shake=0.2)
#print(A.min(),A.max(),B.min(),B.max())
plot_fields(X,Y,A,B,Vx,Vy,Vabs,name+f"{0:04}.png")
time = 0.
tsave = 0.
twrite = 100.
iwrite = 1
while (time < tmax):
    #print(time)
    tsave = tsave + dt
    time = time + dt
    A,B = update_fields(A,B,Vx,Vy,DA,DB,lambA,lambB,f,k,dt,dx,dy)
    if (tsave >= twrite):
        tsave = 0.
        print(time,tsave,twrite,A.min(),A.max(),np.sum(A))
        plot_fields(X,Y,A,B,Vx,Vy,Vabs,name+f"{iwrite:04}.png")
        iwrite = iwrite + 1

### 2

In [ ]:
xmin = 0. 
xmax = 200.
ymin = 0. 
ymax = 200.
nx   = 200
ny   = 200
DA = 0.14
DB = 0.06
f = 0.035
k = 0.065
lambA = 0.0
lambB = 0.0
velx = 0.0
vely = 0.
tmin = 0.
tmax = 10000.
dt   = 1.0
name = 'TWO-2-'

X,Y,dx,dy = create_mesh(xmin,xmax,ymin,ymax,nx,ny)
A,B,Vx,Vy,Vabs = init_fields(X,Y,velx,vely,nx,ny,shake=0.0)
#print(A.min(),A.max(),B.min(),B.max())
plot_fields(X,Y,A,B,Vx,Vy,Vabs,name+f"{0:04}.png")
time = 0.
tsave = 0.
twrite = 100.
iwrite = 1
while (time < tmax):
    #print(time)
    tsave = tsave + dt
    time = time + dt
    A,B = update_fields(A,B,Vx,Vy,DA,DB,lambA,lambB,f,k,dt,dx,dy)
    if (tsave >= twrite):
        tsave = 0.
        print(time,tsave,twrite,A.min(),A.max(),np.sum(A))
        plot_fields(X,Y,A,B,Vx,Vy,Vabs,name+f"{iwrite:04}.png")
        iwrite = iwrite + 1

... done